In [14]:
import csv
import community
import pandas as pd
import numpy as np
import networkx as nx
import math
import matplotlib.pyplot as plt
import pickle
import igraph as ig
import codecs
import operator

## Load Horizon 2020 data into the notebook

Extract Horizon 2020 data from xls files

In [15]:
h2020projects = pd.read_excel (r'cordis-h2020projects.xlsx', sheet_name='project')

In [16]:
h2020organizations = pd.read_excel (r'cordis-h2020organizations.xlsx', sheet_name='organisation')

In [17]:
h2020researchers = pd.read_excel (r'cordis-h2020-msca-fellows.xls', sheet_name='Report 2')

Extract Horizon 2020 projects

In [18]:
#convert dataframe into list
h2020list = list()
h2020namelist = list()
for index, row in h2020projects.iterrows():
    h2020namelist.append(str(row['id']))
    h2020list.append(["H2020project", row['id'], row['acronym'], row['title'], row['startDate'], row['endDate'], row['totalCost'], row['ecMaxContribution']])
len(h2020list)

20878

Extract Horizon 2020 organizations

In [19]:
#convert dataframe into list
h2020orglist = list()
h2020orgnamelist = list()
for index, row in h2020organizations.iterrows():
    h2020orgnamelist.append(str(row['id']))
    h2020orglist.append(["H2020organization", row['projectID'], str(row['id']), row['name'], row['country'] ])
len(h2020orglist)

93434

Extract Horizon 2020 researchers

In [20]:
#convert dataframe into list
h2020reslist = list()
for index, row in h2020researchers.iterrows():
    h2020reslist.append(["H2020researcher", row['projectId'], row['organisationId'], row['firstName'], row['lastName'] ])
len(h2020reslist)

24208

Load previously saved OpenAire publications from pickle

In [21]:
pickle_in = open("totalpublist.pickle","rb")
oapublist = pickle.load(pickle_in)
len(oapublist)

210896

Filter out publications that are connected to H2020 projects

In [22]:
h2020oalist = list()
for pub in oapublist:
    if pub[3] in h2020namelist:
        #print pub[3]
        h2020oalist.append(pub)
len(h2020oalist)

32973

## Network construction

First we populate nodelist with project nodes, then create iGraph object from the nodelist and add edges between nodes based on the connections within lists (e.g. projectID - publicationID)

In [23]:
'''nodelist = list()
edgelist = list()
G = nx.Graph()
for node in h2020list:
    nodelist.append(node)
    G.add_node(node[1], id=node[1], type=node[0], acronym=node[2], title=node[3], startdate=node[4], enddate=node[5], totalCost=node[6], ecContrib=node[7])
'''

'nodelist = list()\nedgelist = list()\nG = nx.Graph()\nfor node in h2020list:\n    nodelist.append(node)\n    G.add_node(node[1], id=node[1], type=node[0], acronym=node[2], title=node[3], startdate=node[4], enddate=node[5], totalCost=node[6], ecContrib=node[7])\n'

Create iGraph object, populate it with nodes from the list of projects

In [24]:
Gig = ig.Graph()
for node in h2020list:
    Gig.add_vertices(1)
    Gig.vs[len(Gig.vs)-1]['name'] = str(node[1])
    Gig.vs[len(Gig.vs)-1]['id'] = str(node[1])
    Gig.vs[len(Gig.vs)-1]['type'] = node[0]
    Gig.vs[len(Gig.vs)-1]['acronym'] = node[2]
    Gig.vs[len(Gig.vs)-1]['title'] = node[3]
    Gig.vs[len(Gig.vs)-1]['startdate'] = node[4]
    Gig.vs[len(Gig.vs)-1]['enddate'] = node[5]
    Gig.vs[len(Gig.vs)-1]['totalCost'] = node[6]
    Gig.vs[len(Gig.vs)-1]['ecContrib'] = node[7]
print len(Gig.vs), "vertices in a graph"

20878 vertices in a graph


Add organisations to the graph, link organisations to projects

In [25]:
#print "nodes before", len(Gig.vs)
for index, node in enumerate(h2020orglist):
    exists = True
    #check if vertice with such name does already exist
    try: 
        Gig.vs.find(str(node[2])).index
    except:
        exists = False #set exists to False
    if exists == False:
        #create new vertice, fill with attributes, connect created vertice to the respective project
        Gig.add_vertices(1)
        Gig.vs[len(Gig.vs)-1]['name'] = str(node[2]) #name attribute is treated specially in iGraph
        Gig.vs[len(Gig.vs)-1]['id'] = str(node[2])
        Gig.vs[len(Gig.vs)-1]['type'] = node[0]
        Gig.vs[len(Gig.vs)-1]['orgname'] = node[3]
        Gig.vs[len(Gig.vs)-1]['country'] = node[4]
        sourceindex = len(Gig.vs)-1
    else: #if vertice with such name already exists
        #find vertice index, connect found vertice to the respective project
        sourceindex = Gig.vs.find(str(node[2])).index
    targetindex = Gig.vs.find(str(node[1])).index
    Gig.add_edges([(sourceindex,targetindex)]) #project to organization    
print len(Gig.vs), "vertices in a graph"
print len(Gig.es), "edges in a graph"

48388 vertices in a graph
93434 edges in a graph


Add researchers to the graph, link them to organisations, if such link is available. If not, link researchers to projects

In [26]:
for index, node in enumerate(h2020reslist):
    #if index % 1000 == 0:
    #    print index
    researcherID = u' '.join((node[3], node[4])).encode('utf-8').strip()
     #check if vertice with such name does already exist
    try: 
        Gig.vs.find(researcherID).index
    except:
        exists = False #set exists to False
    
    if exists == False:
        Gig.add_vertices(1)
        Gig.vs[len(Gig.vs)-1]['name'] = researcherID #name attribute is treated specially in iGraph
        Gig.vs[len(Gig.vs)-1]['id'] = researcherID
        Gig.vs[len(Gig.vs)-1]['type'] = node[0]
        sourceindex = len(Gig.vs)-1
    else:
        sourceindex = Gig.vs.find(researcherID).index
    if pd.isnull(node[2]): #if no organisation available, connect to the project directly
        if str(node[1]) in h2020namelist:
            targetindex = Gig.vs.find(str(node[1])).index
    else: #connect researchers to organisation
        if str(node[2]) in h2020orgnamelist:
            targetindex = Gig.vs.find(str(node[2])).index
    Gig.add_edges([(sourceindex,targetindex)])
print len(Gig.vs), "vertices in a graph"
print len(Gig.es), "edges in a graph"

72596 vertices in a graph
117642 edges in a graph


Add OpenAire publications to the graph, link them to projects

In [ ]:
for index, node in enumerate(h2020oalist):
    #if index % 5000 == 0:
    #    print index   
    try: 
        Gig.vs.find(str(node[0])).index
    except:
        exists = False #set exists to False
        
    if exists == False:
        Gig.add_vertices(1)
        Gig.vs[len(Gig.vs)-1]['name'] = str(node[0]) #name attribute is treated specially in iGraph
        Gig.vs[len(Gig.vs)-1]['id'] = str(node[0])
        Gig.vs[len(Gig.vs)-1]['type'] = "OpenAire Publication"
        Gig.vs[len(Gig.vs)-1]['date'] = node[1]
        Gig.vs[len(Gig.vs)-1]['title'] = node[2]
        Gig.vs[len(Gig.vs)-1]['subtype'] = node[4]
        sourceindex = len(Gig.vs)-1
    else: #if vertice with such name already exists
        #find vertice index, connect found vertice to the respective project
        sourceindex = Gig.vs.find(str(node[0])).index
        
    targetindex = Gig.vs.find(str(node[3])).index #connect publication to project
    Gig.add_edges([(sourceindex,targetindex)]) #publication to project
print len(Gig.vs), "vertices in a graph"
print len(Gig.es), "edges in a graph"

Load/save calculated graph as necessary. To save, uncomment lines 1-3. To load previously saved graph, uncomment lines 4-5

In [419]:
'''pickle_out = open("h2020graph.pickle","wb")
pickle.dump(Gig, pickle_out)
pickle_out.close()'''
'''pickle_in = open("h2020graph.pickle","rb")
Gig = pickle.load(pickle_in)'''

Deal with encoding issues, otherwise will not let you write to graphml. 
Go through each vertice attribute values and encode them to UTF-8

In [366]:
for v in Gig.vs:
    #print v['lastName']
    attributes = ['title']
    #attributes = ['title']
    for attribute in attributes:
        rawvalue = v[attribute]
        if v[attribute]:
            encoded = rawvalue.encode('utf-8').strip()
            v[attribute] = encoded

In [329]:
#save graph
Gig.write_graphml("H2020_igraph_researchers.graphml")

In [230]:
'''pickle_out = open("h2020graph.pickle","wb")
pickle.dump(Gig, pickle_out)
pickle_out.close()'''

## Calculate SNA measures for all nodes of the network

In [330]:
#network density
density = Gig.density(loops=False)

In [331]:
#betweenness centrality
betw = Gig.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)

2829067.0943998573

In [332]:
'''pickle_out = open("betwh2020.pickle","wb")
pickle.dump(betw, pickle_out)
pickle_out.close()'''

'pickle_out = open("betwh2020.pickle","wb")\npickle.dump(betw, pickle_out)\npickle_out.close()'

In [333]:
#eigenvector centrality
eigen = Gig.eigenvector_centrality(directed=False, scale=True, weights=None, return_eigenvalue=False)

In [334]:
'''pickle_out = open("eigenwh2020.pickle","wb")
pickle.dump(eigen, pickle_out)
pickle_out.close()'''

'pickle_out = open("eigenwh2020.pickle","wb")\npickle.dump(eigen, pickle_out)\npickle_out.close()'

In [335]:
#closeness centrality
closeness = Gig.closeness(vertices=None, mode='ALL', cutoff=None, weights=None, normalized=True)

In [336]:
'''pickle_out = open("closenesswh2020.pickle","wb")
pickle.dump(closeness, pickle_out)
pickle_out.close()'''

'pickle_out = open("closenesswh2020.pickle","wb")\npickle.dump(closeness, pickle_out)\npickle_out.close()'

Create a mapping between vertice names and vertice indices (to access centrality values by node ID)

In [337]:
namesGig = Gig.vs['id']
nodesdict = dict()
for name in namesGig:
    nodesdict[name] = namesGig.index(name)
    #print name, nodesdict[name], betw[nodesdict[name]]

Normalize betweennes centrality against maximum value

In [339]:
maxbetw=max(betw)
betwnorm = list()
for index,item in enumerate(betw):
    betwnorm.append(item/maxbetw)

## Reporting results: get top20 organizations by betweenness centrality

In [411]:
resultlist = list()
for node in Gig.vs:
    nodeid = node['id']
    if (node['type'] == 'H2020organization'):
        orgname =  str(node['orgname'])
        node['orgname'] = orgname
        #orgname = u' '.join((orgname, '')).encode('utf-8').strip()
        #node['subtype'] = str(node['orgname']) #for later export to graphml and labeling the nodes in gephi
        resultlist.append([node['country'], node['orgname'], betwnorm[nodesdict[nodeid]], node.degree()])
        #print resultlist[len(resultlist)-1]
#sort resulted list of organizations by BetwCent (for reporting)
sortedlist = sorted(resultlist, key=operator.itemgetter(2), reverse=True)
sortedlist

[[u'FR', "u'CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE CNRS'", 1.0, 1078],
 [u'DE',
  "u'FRAUNHOFER GESELLSCHAFT ZUR FOERDERUNG DER ANGEWANDTEN FORSCHUNG E.V.'",
  0.7299273783651884,
  714],
 [u'IT', "u'CONSIGLIO NAZIONALE DELLE RICERCHE'", 0.4855628628421361, 538],
 [u'ES',
  "u'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS'",
  0.4606399463651525,
  542],
 [u'FR',
  "u'COMMISSARIAT A L ENERGIE ATOMIQUE ET AUX ENERGIES ALTERNATIVES'",
  0.38100312884025694,
  453],
 [u'UK',
  "u'THE CHANCELLOR, MASTERS AND SCHOLARS OF THE UNIVERSITY OF OXFORD'",
  0.2693592111031605,
  482],
 [u'BE', "u'KATHOLIEKE UNIVERSITEIT LEUVEN'", 0.26598876003193955, 490],
 [u'UK',
  "u'THE CHANCELLOR MASTERS AND SCHOLARS OF THE UNIVERSITY OF CAMBRIDGE'",
  0.2540454635675907,
  497],
 [u'UK',
  "u'IMPERIAL COLLEGE OF SCIENCE TECHNOLOGY AND MEDICINE'",
  0.23156123090596117,
  387],
 [u'DK', "u'KOBENHAVNS UNIVERSITET'", 0.22886631676220623, 481],
 [u'DK', "u'DANMARKS TEKNISKE UNIVERSITET'",

In [400]:
dfObj = pd.DataFrame(sortedlist, columns = ['Country','Organisation','Betwenness Centrality','Degree']) 
dfObj.to_clipboard(sep=',')
dfObj

,Country,Organisation,Betwenness Centrality,Degree
0,FR,"'\'""u\\\'CENTRE NATIONAL DE LA RECHERCHE SCIEN...",1.000000,1078
1,DE,"'\'""u\\\'FRAUNHOFER GESELLSCHAFT ZUR FOERDERUN...",0.729927,714
2,IT,"'\'""u\\\'CONSIGLIO NAZIONALE DELLE RICERCHE\\\...",0.485563,538
3,ES,"'\'""u\\\'AGENCIA ESTATAL CONSEJO SUPERIOR DE I...",0.460640,542
4,FR,"'\'""u\\\'COMMISSARIAT A L ENERGIE ATOMIQUE ET ...",0.381003,453
5,UK,"'\'""u\\\'THE CHANCELLOR, MASTERS AND SCHOLARS ...",0.269359,482
6,BE,"'\'""u\\\'KATHOLIEKE UNIVERSITEIT LEUVEN\\\'""\''",0.265989,490
7,UK,"'\'""u\\\'THE CHANCELLOR MASTERS AND SCHOLARS O...",0.254045,497
8,UK,"'\'""u\\\'IMPERIAL COLLEGE OF SCIENCE TECHNOLOG...",0.231561,387
9,DK,"'\'""u\\\'KOBENHAVNS UNIVERSITET\\\'""\''",0.228866,481


## Reporting results: get top20 projects by betweenness centrality

In [412]:
#FOR REPORT: get top20 projects by betweenness centrality, and then by BF ratio
resultlist = list()
for node in Gig.vs:
    nodeid = node['id']
    #if (betwnorm[nodesdict[nodeid]] > 0.1) and (node['type'] == 'H2020organization'):
    #    print node['name'], betwnorm[nodesdict[nodeid]]
    node['betwcent'] = betwnorm[nodesdict[nodeid]]
    if (node['type'] == 'H2020project'):
        node['ratio'] = betwnorm[nodesdict[nodeid]] / node['ecContrib']
        resultlist.append([node['acronym'], node['title'], betwnorm[nodesdict[nodeid]], node.degree(), node['ecContrib'], node['ratio']])
        #print resultlist[len(resultlist)-1]
#sort resulted list of projects by BetwCent (for reporting)
sortedlist = sorted(resultlist, key=operator.itemgetter(2), reverse=True)
sortedlist

[[u'HBP SGA1',
  u'Human Brain Project Specific Grant Agreement 1',
  0.506912729346578,
  416,
  89000000.0,
  5.695648644343573e-09],
 [u'GrapheneCore1',
  u'Graphene-based disruptive technologies',
  0.3914516068889744,
  330,
  89000000.0,
  4.398332661673869e-09],
 [u'GrapheneCore2',
  u'Graphene Flagship Core Project 2',
  0.285183922510148,
  146,
  88000000.0,
  3.240726392160773e-09],
 [u'EUROfusion',
  u'Implementation of activities described in the Roadmap to Fusion during Horizon 2020 through a Joint programme of the members of the EUROfusion consortium',
  0.19532491433918925,
  928,
  440800000.0,
  4.4311459695823334e-10],
 [u'InvisiblesPlus',
  u'InvisiblesPlus',
  0.08082519205977198,
  197,
  2070000.0,
  3.904598650230531e-08],
 [u'AIDA-2020',
  u'Advanced European Infrastructures for Detectors at Accelerators',
  0.08005906925594704,
  137,
  10000000.0,
  8.005906925594704e-09],
 [u'ELIXIR-EXCELERATE',
  u'ELIXIR-EXCELERATE: Fast-track ELIXIR implementation and dri

In [347]:
dfObj = pd.DataFrame(sortedlist, columns = ['Project acronym','Project title','Betweenness Centrality','Degree','EC contribution','BFR score']) 
dfObj.to_clipboard(sep=',')
dfObj

,Project acronym,Project title,Betwenness Centrality,Degree,EC contribution,BFR score
0,HBP SGA1,Human Brain Project Specific Grant Agreement 1,0.506913,416,8.900000e+07,5.695649e-09
1,GrapheneCore1,Graphene-based disruptive technologies,0.391452,330,8.900000e+07,4.398333e-09
2,GrapheneCore2,Graphene Flagship Core Project 2,0.285184,146,8.800000e+07,3.240726e-09
3,EUROfusion,Implementation of activities described in the ...,0.195325,928,4.408000e+08,4.431146e-10
4,InvisiblesPlus,InvisiblesPlus,0.080825,197,2.070000e+06,3.904599e-08
5,AIDA-2020,Advanced European Infrastructures for Detector...,0.080059,137,1.000000e+07,8.005907e-09
6,ELIXIR-EXCELERATE,ELIXIR-EXCELERATE: Fast-track ELIXIR implement...,0.071614,113,1.905148e+07,3.758986e-09
7,ECOPOTENTIAL,ECOPOTENTIAL: IMPROVING FUTURE ECOSYSTEM BENEF...,0.061253,117,1.487434e+07,4.118005e-09
8,ENSAR2,European Nuclear Science and Application Resea...,0.058099,171,1.000000e+07,5.809878e-09
9,OpenAIRE2020,Open Access Infrastructure for Research in Eur...,0.048343,104,1.300000e+07,3.718668e-09


In [413]:
#save graph
import sys
reload(sys)
sys.setdefaultencoding('utf8')
Gig.write_graphml("H2020_igraph_bc_ratio_final.graphml")

## Eigen Vector Centrality

In [415]:
#normalize centrality against max
maxeigen=max(eigen)
eigennorm = list()
for index,item in enumerate(eigen):
    eigennorm.append(item/maxeigen)

## Reporting results: get top20 organizations by eigenvector centrality

In [420]:
#FOR REPORT: get top20 organizations by eigenvector centrality
resultlist = list()
for node in Gig.vs:
    nodeid = node['id']
    if (node['type'] == 'H2020organization'):
        resultlist.append([node['country'], node['orgname'], eigennorm[nodesdict[nodeid]], node.degree()])
#sort resulted list of organizations by BetwCent (for reporting)
sortedlist = sorted(resultlist, key=operator.itemgetter(2), reverse=True)
sortedlist

[[u'FR', u'CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE CNRS', 1.0, 1078],
 [u'DE',
  u'FRAUNHOFER GESELLSCHAFT ZUR FOERDERUNG DER ANGEWANDTEN FORSCHUNG E.V.',
  0.4098853701514245,
  714],
 [u'FR',
  u'COMMISSARIAT A L ENERGIE ATOMIQUE ET AUX ENERGIES ALTERNATIVES',
  0.307901531510869,
  453],
 [u'IT', u'CONSIGLIO NAZIONALE DELLE RICERCHE', 0.2825251332877465, 538],
 [u'ES',
  u'AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGACIONES CIENTIFICAS',
  0.26758544704062837,
  542],
 [u'CH',
  u'EIDGENOESSISCHE TECHNISCHE HOCHSCHULE ZUERICH',
  0.15024294134497823,
  336],
 [u'DE', u'KARLSRUHER INSTITUT FUER TECHNOLOGIE', 0.13665394586106094, 210],
 [u'DK', u'DANMARKS TEKNISKE UNIVERSITET', 0.13546002244482205, 359],
 [u'UK',
  u'THE CHANCELLOR, MASTERS AND SCHOLARS OF THE UNIVERSITY OF OXFORD',
  0.135431741321396,
  482],
 [u'FI', u'Teknologian tutkimuskeskus VTT Oy', 0.1283060268336969, 247],
 [u'CH',
  u'ECOLE POLYTECHNIQUE FEDERALE DE LAUSANNE',
  0.12700207454456083,
  384],
 [u'UK',
  

In [421]:
dfObj = pd.DataFrame(sortedlist, columns = ['Country','Organisation','Eigenvector Centrality','Degree']) 
dfObj.to_clipboard(sep=',')
dfObj

,Country,Organisation,Eigenvector Centrality,Degree
0,FR,CENTRE NATIONAL DE LA RECHERCHE SCIENTIFIQUE CNRS,1.000000e+00,1078
1,DE,FRAUNHOFER GESELLSCHAFT ZUR FOERDERUNG DER ANG...,4.098854e-01,714
2,FR,COMMISSARIAT A L ENERGIE ATOMIQUE ET AUX ENERG...,3.079015e-01,453
3,IT,CONSIGLIO NAZIONALE DELLE RICERCHE,2.825251e-01,538
4,ES,AGENCIA ESTATAL CONSEJO SUPERIOR DE INVESTIGAC...,2.675854e-01,542
5,CH,EIDGENOESSISCHE TECHNISCHE HOCHSCHULE ZUERICH,1.502429e-01,336
6,DE,KARLSRUHER INSTITUT FUER TECHNOLOGIE,1.366539e-01,210
7,DK,DANMARKS TEKNISKE UNIVERSITET,1.354600e-01,359
8,UK,"THE CHANCELLOR, MASTERS AND SCHOLARS OF THE UN...",1.354317e-01,482
9,FI,Teknologian tutkimuskeskus VTT Oy,1.283060e-01,247


## Reporting results: get top20 projects by eigenvector centrality

In [423]:
#FOR REPORT: get top20 projects by eigenvector centrality, and then by BF ratio
resultlist = list()
for node in Gig.vs:
    nodeid = node['id']
    #if (betwnorm[nodesdict[nodeid]] > 0.1) and (node['type'] == 'H2020organization'):
    #    print node['name'], betwnorm[nodesdict[nodeid]]
    node['eigencent'] = eigennorm[nodesdict[nodeid]]
    if (node['type'] == 'H2020project'):
        node['ratio'] = eigennorm[nodesdict[nodeid]] / node['ecContrib']
        resultlist.append([node['acronym'], node['title'], eigennorm[nodesdict[nodeid]], node.degree(), node['ecContrib'], node['ratio']])
        #print resultlist[len(resultlist)-1]
#sort resulted list of projects by BetwCent (for reporting)
sortedlist = sorted(resultlist, key=operator.itemgetter(2), reverse=True)
sortedlist

[[u'GrapheneCore1',
  u'Graphene-based disruptive technologies',
  0.23560175831133282,
  330,
  89000000.0,
  2.6472107675430655e-09],
 [u'HBP SGA1',
  u'Human Brain Project Specific Grant Agreement 1',
  0.21539996165701739,
  416,
  89000000.0,
  2.4202242882810943e-09],
 [u'GrapheneCore2',
  u'Graphene Flagship Core Project 2',
  0.1885150279724045,
  146,
  88000000.0,
  2.142216226959142e-09],
 [u'EUROfusion',
  u'Implementation of activities described in the Roadmap to Fusion during Horizon 2020 through a Joint programme of the members of the EUROfusion consortium',
  0.1178592555656584,
  928,
  440800000.0,
  2.673758066371561e-10],
 [u'EOSCpilot',
  u'The European Open Science Cloud for Research Pilot Project.',
  0.10334941271190011,
  49,
  9953067.5,
  1.0383674451308615e-08],
 [u'EOSC-hub',
  u'Integrating and managing services for the European Open Science Cloud',
  0.10007196251312635,
  77,
  30000000.0,
  3.3357320837708784e-09],
 [u'AIDA-2020',
  u'Advanced European 

In [424]:
dfObj = pd.DataFrame(sortedlist, columns = ['Project acronym','Project title','EigenVector Centrality','Degree','EC contribution','CFR score']) 
dfObj.to_clipboard(sep=',')
dfObj

,Project acronym,Project title,EigenVector Centrality,Degree,EC contribution,CFR score
0,GrapheneCore1,Graphene-based disruptive technologies,2.356018e-01,330,8.900000e+07,2.647211e-09
1,HBP SGA1,Human Brain Project Specific Grant Agreement 1,2.154000e-01,416,8.900000e+07,2.420224e-09
2,GrapheneCore2,Graphene Flagship Core Project 2,1.885150e-01,146,8.800000e+07,2.142216e-09
3,EUROfusion,Implementation of activities described in the ...,1.178593e-01,928,4.408000e+08,2.673758e-10
4,EOSCpilot,The European Open Science Cloud for Research P...,1.033494e-01,49,9.953068e+06,1.038367e-08
5,EOSC-hub,Integrating and managing services for the Euro...,1.000720e-01,77,3.000000e+07,3.335732e-09
6,AIDA-2020,Advanced European Infrastructures for Detector...,9.570348e-02,137,1.000000e+07,9.570348e-09
7,ECOPOTENTIAL,ECOPOTENTIAL: IMPROVING FUTURE ECOSYSTEM BENEF...,8.530446e-02,117,1.487434e+07,5.735008e-09
8,ELIXIR-EXCELERATE,ELIXIR-EXCELERATE: Fast-track ELIXIR implement...,8.205765e-02,113,1.905148e+07,4.307153e-09
9,ARIES,Accelerator Research and Innovation for Europe...,8.051104e-02,58,1.000000e+07,8.051104e-09
